**Stephanie Beaver**
**Capstone Project**
**Working code completed on May 30 2025**
Scrapes google maps reviews of all stores in regions that the search is for. deposits them in HTML format into a temporary colab folder. Tracks which individual stores (URLs) are successfully scraped.


In [ ]:
import sys
!{sys.executable} -m pip install selenium


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.14.0
    Uninstalling typing_extensions-4.14.0:
      Successfully uninstalled typing_extensions-4.14.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 r

In [ ]:
pip install dateparser

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 9.9 MB/s eta 0:00:00


In [ ]:
from datetime import datetime, timedelta
import random
import time
import tempfile
import re
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:

# --- CREATE UNIQUE TEMP PROFILE FOR COLAB ---
temp_profile_dir = tempfile.mkdtemp()
print(f"Using temporary Chrome profile at: {temp_profile_dir}")

# --- FUNCTION TO ENFORCE ENGLISH URLs ---
def enforce_english_url(url):
    if url is None:
        return None
    if 'hl=' in url:
        url = re.sub(r'hl=[^&]*', 'hl=en', url)
    else:
        if '?' in url:
            url += '&hl=en'
        else:
            url += '?hl=en'
    return url


Using temporary Chrome profile at: /tmp/tmp1o07lg7l


In [ ]:
## Search the store or company inside"Double double quotations" exactly as found on google reviews
## To search more than on area, include a , "...." for as many URLs as necessary. Don't scroll too far in or too far out
## Will not recognize if there are only one or two stores in the URL

regional_search_urls = [
    "https://www.google.com/maps/search/%22sportsman's+Warehouse%22+Arkansas/@35.1006432,-93.048218,8z?entry=ttu&g_ep=EgoyMDI1MDYxMS4wIKXMDSoASAFQAw%3D%3D",

]



In [ ]:
 def dynamic_sleep(review_count):
    if review_count < 500:
        sleep_time = random.uniform(3, 5)
    elif review_count < 1000:
        sleep_time = random.uniform(5, 10)
    elif review_count < 1700:
        sleep_time = random.uniform(10, 15)
    elif review_count < 2000:
        sleep_time = random.uniform(15, 25)
    else:
        sleep_time = random.uniform(15, 25)
    print(f"Sleeping for {sleep_time:.2f}s based on {review_count} reviews loaded")
    time.sleep(sleep_time)

# --- SETUP CHROME OPTIONS ---
chrome_options = Options()
chrome_options.add_argument(f"--user-data-dir={temp_profile_dir}")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument('--headless=new')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument("--remote-debugging-port=9222")


# --- INITIALIZE CHROME DRIVER ---
driver = webdriver.Chrome(options=chrome_options)
wait = WebDriverWait(driver, 20)

# --- LOAD EXISTING SCRAPED URLS ---
scraped_log_path = "/content/sample_data/scraped_urls.txt"
try:
    with open(scraped_log_path, "r", encoding="utf-8") as log_file:
        already_scraped = set(line.strip() for line in log_file.readlines())
except FileNotFoundError:
    already_scraped = set()

# --- ENSURE HTML OUTPUT FILE EXISTS ---
output_file = "/content/sample_data/sportsmans_scraped.html"
if not os.path.exists(output_file):
    open(output_file, "w", encoding="utf-8").close()  # create empty file if not there



# --- STARTING URL ---
store_urls = set()
print("Starting multi-region scraping...")


for region_url in regional_search_urls:
    print(f"\n🌎 Navigating to region: {region_url}")
    driver.get(region_url)
    time.sleep(random.uniform(40, 50))

    try:
        side_panel = wait.until(EC.presence_of_element_located((By.XPATH, "//div[@role='feed']")))
    except Exception as e:
        print(f"⚠️ Could not find store list panel for {region_url}: {e}")
        continue

    scroll_increment = 1000
    max_attempts = 1000
    no_change_counter = 0
    max_no_change_allowed = 3
    last_scroll_pos = driver.execute_script("return arguments[0].scrollTop", side_panel)

    print(f"📜 Scrolling through results for: {region_url}")
    for attempt in range(max_attempts):
        driver.execute_script("arguments[0].scrollTop += arguments[1]", side_panel, scroll_increment)
        sleep_time = random.choice([random.uniform(5, 7), random.uniform(5, 10), random.uniform(10, 20)])
        print(f"Scroll {attempt+1}, sleeping {sleep_time:.2f}s")
        time.sleep(sleep_time)

        new_scroll_pos = driver.execute_script("return arguments[0].scrollTop", side_panel)
        if new_scroll_pos == last_scroll_pos:
            no_change_counter += 1
            if no_change_counter >= max_no_change_allowed:
                print("🛑 No more new scroll detected.")
                break
        else:
            no_change_counter = 0
            last_scroll_pos = new_scroll_pos

        links = driver.find_elements(By.CSS_SELECTOR, "a.hfpxzc")
        for link in links:
            href = link.get_attribute('href')
            if href:
                store_urls.add(enforce_english_url(href))

    print(f"✅ Done with region: {region_url} — Total unique stores so far: {len(store_urls)}")


# --- REVIEW SCRAPING FOR EACH STORE ---
store_locations = {}

for i, store_url in enumerate(store_urls, start=1):
    if store_url in already_scraped:
        print(f"⏭️  Skipping already-scraped store: {store_url}")
        continue

    print(f"\n🎯 Visiting store {i}/{len(store_urls)}: {store_url}")
    try:
        driver.get(store_url)
        time.sleep(random.uniform(5, 10))

        try:
            address_element = wait.until(EC.presence_of_element_located(
                (By.XPATH, '//div[contains(@class, "Io6YTe") and contains(@class, "fontBodyMedium")]')
            ))
            store_location = address_element.text.strip()
            print(f"Store location found: {store_location}")
            store_locations[store_url] = store_location
        except Exception as e:
            print(f"⚠️ Could not find store location info: {e}")
            store_location = "Unknown"
            store_locations[store_url] = store_location

        reviews_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(@aria-label, 'reviews')]")))
        reviews_button.click()
        time.sleep(random.uniform(4, 6))

        sort_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Sort reviews"]')))
        sort_button.click()
        time.sleep(random.uniform(4, 6))

        newest_option = wait.until(EC.element_to_be_clickable((By.XPATH, '//div[@role="menuitemradio" and .="Newest"]')))
        newest_option.click()
        time.sleep(random.uniform(4, 6))

        review_panel = wait.until(
            EC.presence_of_element_located(
                (By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]')
            )
        )
        last_height = driver.execute_script("return arguments[0].scrollHeight", review_panel)

        scroll_attempts = 0
        max_scroll_attempts = 1000
        no_scroll_change_limit = 5

        while scroll_attempts < max_scroll_attempts:
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", review_panel)
            review_blocks = review_panel.find_elements(By.XPATH, './/div[contains(@class, "jftiEf")]')
            review_count = len(review_blocks)
            dynamic_sleep(review_count)

            new_height = driver.execute_script("return arguments[0].scrollHeight", review_panel)
            if new_height == last_height:
                scroll_attempts += 1
                if scroll_attempts >= no_scroll_change_limit:
                    print("🔚 No more reviews to load.")
                    break
            else:
                last_height = new_height
                scroll_attempts = 0

        print(f"📝 Reviews loaded: {len(review_blocks)}")

        more_buttons = driver.find_elements(By.XPATH, "//button[contains(text(), 'More')]")
        print(f"Found {len(more_buttons)} 'More' buttons to expand.")
        for btn in more_buttons:
            try:
                driver.execute_script("arguments[0].click();", btn)
                time.sleep(random.uniform(1, 2))
            except Exception as e:
                print(f"⚠️ Failed to click 'More' button: {e}")
                continue

        review_blocks = review_panel.find_elements(By.XPATH, './/div[contains(@class, "jftiEf")]')

        # --- Build and append current store's reviews HTML ---
        store_html = f'\n<!-- Store URL: {store_url} -->\n'
        store_html += f'<!-- Store Location: {store_locations.get(store_url, "Unknown")} -->\n'
        for review in review_blocks:
            html = review.get_attribute('outerHTML')
            store_html += html + "\n"

        with open(output_file, "a", encoding="utf-8") as f:
            f.write(store_html)

        print(f"✅ Collected {len(review_blocks)} reviews from store {i}")
        print(f"💾 Saved progress after store {i}: {store_locations.get(store_url, 'Unknown')}")

        # Log this store as scraped
        with open(scraped_log_path, "a", encoding="utf-8") as log_file:
            log_file.write(store_url + "\n")

    except Exception as e:
        print(f"❌ Error scraping store {store_url}: {e}")
        continue

print(f"\n✨ Finished scraping! All reviews saved to '{output_file}'")
print("🧹 Quitting browser...")
driver.quit()


Starting multi-region scraping...

🌎 Navigating to region: https://www.google.com/maps/search/%22Sportsman's+Warehouse%22+Alaska/@61.0851128,-149.9888905,10z?authuser=0&hl=en&entry=ttu&g_ep=EgoyMDI1MDYwOS4xIKXMDSoASAFQAw%3D%3D
⚠️ Could not find store list panel for https://www.google.com/maps/search/%22Sportsman's+Warehouse%22+Alaska/@61.0851128,-149.9888905,10z?authuser=0&hl=en&entry=ttu&g_ep=EgoyMDI1MDYwOS4xIKXMDSoASAFQAw%3D%3D: Message: 
Stacktrace:
#0 0x5c1078eaac9a <unknown>
#1 0x5c10789506e0 <unknown>
#2 0x5c10789a2117 <unknown>
#3 0x5c10789a2311 <unknown>
#4 0x5c10789f0ec4 <unknown>
#5 0x5c10789c7e5d <unknown>
#6 0x5c10789ee2cc <unknown>
#7 0x5c10789c7c03 <unknown>
#8 0x5c107899447b <unknown>
#9 0x5c10789950e1 <unknown>
#10 0x5c1078e6f44b <unknown>
#11 0x5c1078e7337f <unknown>
#12 0x5c1078e56f89 <unknown>
#13 0x5c1078e73f18 <unknown>
#14 0x5c1078e3b6df <unknown>
#15 0x5c1078e98308 <unknown>
#16 0x5c1078e984e6 <unknown>
#17 0x5c1078ea9b76 <unknown>
#18 0x7e61cf02bac3 <unknown>




KeyboardInterrupt: 

In [ ]:
###For future use- use this section in place of the previous one
## This one has the date set up so that it will only collect reviews less than 1 month ago...
# --- STARTING URL WITH WIDE ZOOM ---
search_url = "https://www.google.com/maps/search/sportsmans+warehouse/@39.8283,-119.6732484,3z?hl=en%22&entry=ttu&g_ep=EgoyMDI1MDYwMS4wIKXMDSoASAFQAw%3D%3D"
driver.get(search_url)
time.sleep(random.uniform(5, 10))

# --- SCROLL THROUGH STORE LIST TO COLLECT STORE URLs ---
side_panel = wait.until(EC.presence_of_element_located((By.XPATH, "//div[@role='feed']")))
scroll_increment = 1000
max_attempts = 1000
no_change_counter = 0
max_no_change_allowed = 3
last_scroll_pos = driver.execute_script("return arguments[0].scrollTop", side_panel)

store_urls = set()
print("Scrolling through store list...")

for attempt in range(max_attempts):
    driver.execute_script("arguments[0].scrollTop += arguments[1]", side_panel, scroll_increment)
    sleep_time = random.choice([random.uniform(5, 10), random.uniform(10, 20), random.uniform(20, 40)])
    print(f"Scroll {attempt+1}, sleeping {sleep_time:.2f}s")
    time.sleep(sleep_time)

    new_scroll_pos = driver.execute_script("return arguments[0].scrollTop", side_panel)
    if new_scroll_pos == last_scroll_pos:
        no_change_counter += 1
        if no_change_counter >= max_no_change_allowed:
            break
    else:
        no_change_counter = 0
        last_scroll_pos = new_scroll_pos

    links = driver.find_elements(By.CSS_SELECTOR, "a.hfpxzc")
    for link in links:
        href = link.get_attribute('href')
        if href:
            store_urls.add(enforce_english_url(href))

    print(f"Unique stores found so far: {len(store_urls)}")

# --- REVIEW SCRAPING FOR EACH STORE -Added the date limits to go back one month can change dates from 30 to whatever number of days you need
store_locations = {}
one_month_ago = datetime.now() - timedelta(days=30)

for i, store_url in enumerate(store_urls, start=1):
    if store_url in already_scraped:
        print(f"⏭️  Skipping already-scraped store: {store_url}")
        continue

    print(f"\n🎯 Visiting store {i}/{len(store_urls)}: {store_url}")
    try:
        driver.get(store_url)
        time.sleep(random.uniform(5, 10))

        try:
            address_element = wait.until(EC.presence_of_element_located(
                (By.XPATH, '//div[contains(@class, "Io6YTe") and contains(@class, "fontBodyMedium")]')
            ))
            store_location = address_element.text.strip()
            print(f"Store location found: {store_location}")
            store_locations[store_url] = store_location
        except Exception as e:
            print(f"⚠️ Could not find store location info: {e}")
            store_location = "Unknown"
            store_locations[store_url] = store_location

        reviews_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(@aria-label, 'reviews')]")))
        reviews_button.click()
        time.sleep(random.uniform(4, 6))

        sort_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Sort reviews"]')))
        sort_button.click()
        time.sleep(random.uniform(4, 6))

        newest_option = wait.until(EC.element_to_be_clickable((By.XPATH, '//div[@role="menuitemradio" and .="Newest"]')))
        newest_option.click()
        time.sleep(random.uniform(4, 6))

        review_panel = wait.until(
            EC.presence_of_element_located(
                (By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]')
            )
        )
        last_height = driver.execute_script("return arguments[0].scrollHeight", review_panel)

        scroll_attempts = 0
        max_scroll_attempts = 1000
        no_scroll_change_limit = 5

        while scroll_attempts < max_scroll_attempts:
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", review_panel)
            review_blocks = review_panel.find_elements(By.XPATH, './/div[contains(@class, "jftiEf")]')
            review_count = len(review_blocks)
            dynamic_sleep(review_count)

            new_height = driver.execute_script("return arguments[0].scrollHeight", review_panel)
            if new_height == last_height:
                scroll_attempts += 1
                if scroll_attempts >= no_scroll_change_limit:
                    print("🔚 No more reviews to load.")
                    break
            else:
                last_height = new_height
                scroll_attempts = 0

        more_buttons = driver.find_elements(By.XPATH, "//button[contains(text(), 'More')]")
        for btn in more_buttons:
            try:
                driver.execute_script("arguments[0].click();", btn)
                time.sleep(random.uniform(1, 2))
            except Exception as e:
                continue

        review_blocks = review_panel.find_elements(By.XPATH, './/div[contains(@class, "jftiEf")]')
        filtered_reviews_html = ""
        valid_review_count = 0

        for review in review_blocks:
            try:
                review_html = review.get_attribute("outerHTML")
                date_element = review.find_element(By.XPATH, './/span[contains(@class, "rsqaWe")]')
                date_text = date_element.text.strip()
                parsed_date = dateparser.parse(date_text)
                if parsed_date and parsed_date >= one_month_ago:
                    filtered_reviews_html += review_html + "\n"
                    valid_review_count += 1
            except Exception as e:
                continue

        store_html = f'\n<!-- Store URL: {store_url} -->\n'
        store_html += f'<!-- Store Location: {store_locations.get(store_url, "Unknown")} -->\n'
        store_html += filtered_reviews_html

        with open(output_file, "a", encoding="utf-8") as f:
            f.write(store_html)

        print(f"✅ Appended {valid_review_count} reviews from past month.")
        with open(scraped_log_path, "a", encoding="utf-8") as log_file:
            log_file.write(store_url + "\n")

    except Exception as e:
        print(f"❌ Error scraping store {store_url}: {e}")
        continue

print(f"\n✨ Finished scraping! All reviews saved to '{output_file}'")
print("🧹 Quitting browser...")
driver.quit()


In [ ]:
import textwrap

In [ ]:
 def dynamic_sleep(review_count):
    if review_count < 500:
        sleep_time = random.uniform(3, 5)
    elif review_count < 1000:
        sleep_time = random.uniform(5, 10)
    elif review_count < 1700:
        sleep_time = random.uniform(10, 15)
    elif review_count < 2000:
        sleep_time = random.uniform(15, 25)
    else:
        sleep_time = random.uniform(15, 25)
    print(f"Sleeping for {sleep_time:.2f}s based on {review_count} reviews loaded")
    time.sleep(sleep_time)

# --- SETUP CHROME OPTIONS ---
chrome_options = Options()
chrome_options.add_argument(f"--user-data-dir={temp_profile_dir}")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument('--headless=new')
chrome_options.add_argument('--disable-gpu')

# --- INITIALIZE CHROME DRIVER ---
driver = webdriver.Chrome(options=chrome_options)
wait = WebDriverWait(driver, 20)

# --- LOAD EXISTING SCRAPED URLS ---
scraped_log_path = "/content/sample_data/scraped_urls.txt"
try:
    with open(scraped_log_path, "r", encoding="utf-8") as log_file:
        already_scraped = set(line.strip() for line in log_file.readlines())
except FileNotFoundError:
    already_scraped = set()

# --- ENSURE HTML OUTPUT FILE EXISTS ---
output_file = "/content/sample_data/sportsmans_scraped.html"
if not os.path.exists(output_file):
    open(output_file, "w", encoding="utf-8").close()  # create empty file if not there

# --- STARTING URL ---
store_urls = set()
print("Starting multi-region scraping...")

for region_url in regional_search_urls:
    print(f"\n🌎 Navigating to region: {region_url}")
    driver.get(region_url)
    time.sleep(random.uniform(5, 10))

try:
    # Try getting the scrollable panel using a more robust reference
    links_panel = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "div[aria-label][jslog]"))
    )
except Exception as e:
    print(f"⚠️ Still no scrollable container found in: {region_url}")
    with open("failed_region_dump.html", "w", encoding="utf-8") as f:
        f.write(driver.page_source)
    continue


    print(f"📜 Scrolling and extracting store URLs for: {region_url}")

    scroll_increment = 1000
    max_attempts = 1000
    no_change_counter = 0
    max_no_change_allowed = 3
    last_scroll_pos = driver.execute_script("return arguments[0].scrollTop", side_panel)

    for attempt in range(max_attempts):
        driver.execute_script("arguments[0].scrollTop += arguments[1]", side_panel, scroll_increment)
        sleep_time = random.choice([random.uniform(3, 5), random.uniform(5, 10), random.uniform(10, 20)])
        print(f"Scroll {attempt+1}, sleeping {sleep_time:.2f}s")
        time.sleep(sleep_time)

        new_scroll_pos = driver.execute_script("return arguments[0].scrollTop", side_panel)
        if new_scroll_pos == last_scroll_pos:
            no_change_counter += 1
            if no_change_counter >= max_no_change_allowed:
                print("🛑 No more new scroll detected.")
                break
        else:
            no_change_counter = 0
            last_scroll_pos = new_scroll_pos

    links = driver.find_elements(By.CSS_SELECTOR, "a.hfpxzc")
    new_links = 0
    for link in links:
        href = link.get_attribute('href')
        if href and enforce_english_url(href) not in store_urls:
            store_urls.add(enforce_english_url(href))
            new_links += 1

    print(f"✅ Region done: {region_url}")
    print(f"➕ Found {new_links} new store URLs — total so far: {len(store_urls)}")


# --- REVIEW SCRAPING FOR EACH STORE ---
store_locations = {}

for i, store_url in enumerate(store_urls, start=1):
    if store_url in already_scraped:
        print(f"⏭️  Skipping already-scraped store: {store_url}")
        continue

    print(f"\n🎯 Visiting store {i}/{len(store_urls)}: {store_url}")
    try:
        driver.get(store_url)
        time.sleep(random.uniform(5, 10))

        try:
            address_element = wait.until(EC.presence_of_element_located(
                (By.XPATH, '//div[contains(@class, "Io6YTe") and contains(@class, "fontBodyMedium")]')
            ))
            store_location = address_element.text.strip()
            print(f"Store location found: {store_location}")
            store_locations[store_url] = store_location
        except Exception as e:
            print(f"⚠️ Could not find store location info: {e}")
            store_location = "Unknown"
            store_locations[store_url] = store_location

        reviews_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(@aria-label, 'reviews')]")))
        reviews_button.click()
        time.sleep(random.uniform(4, 6))

        sort_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Sort reviews"]')))
        sort_button.click()
        time.sleep(random.uniform(4, 6))

        newest_option = wait.until(EC.element_to_be_clickable((By.XPATH, '//div[@role="menuitemradio" and .="Newest"]')))
        newest_option.click()
        time.sleep(random.uniform(4, 6))

        review_panel = wait.until(
            EC.presence_of_element_located(
                (By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]')
            )
        )
        last_height = driver.execute_script("return arguments[0].scrollHeight", review_panel)

        scroll_attempts = 0
        max_scroll_attempts = 1000
        no_scroll_change_limit = 5

        while scroll_attempts < max_scroll_attempts:
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", review_panel)
            review_blocks = review_panel.find_elements(By.XPATH, './/div[contains(@class, "jftiEf")]')
            review_count = len(review_blocks)
            dynamic_sleep(review_count)

            new_height = driver.execute_script("return arguments[0].scrollHeight", review_panel)
            if new_height == last_height:
                scroll_attempts += 1
                if scroll_attempts >= no_scroll_change_limit:
                    print("🔚 No more reviews to load.")
                    break
            else:
                last_height = new_height
                scroll_attempts = 0

        print(f"📝 Reviews loaded: {len(review_blocks)}")

        more_buttons = driver.find_elements(By.XPATH, "//button[contains(text(), 'More')]")
        print(f"Found {len(more_buttons)} 'More' buttons to expand.")
        for btn in more_buttons:
            try:
                driver.execute_script("arguments[0].click();", btn)
                time.sleep(random.uniform(1, 2))
            except Exception as e:
                print(f"⚠️ Failed to click 'More' button: {e}")
                continue

        review_blocks = review_panel.find_elements(By.XPATH, './/div[contains(@class, "jftiEf")]')

        # --- Build and append current store's reviews HTML ---
        store_html = f'\n<!-- Store URL: {store_url} -->\n'
        store_html += f'<!-- Store Location: {store_locations.get(store_url, "Unknown")} -->\n'
        for review in review_blocks:
            html = review.get_attribute('outerHTML')
            store_html += html + "\n"

        with open(output_file, "a", encoding="utf-8") as f:
            f.write(store_html)

        print(f"✅ Collected {len(review_blocks)} reviews from store {i}")
        print(f"💾 Saved progress after store {i}: {store_locations.get(store_url, 'Unknown')}")

        # Log this store as scraped
        with open(scraped_log_path, "a", encoding="utf-8") as log_file:
            log_file.write(store_url + "\n")

    except Exception as e:
        print(f"❌ Error scraping store {store_url}: {e}")
        continue

print(f"\n✨ Finished scraping! All reviews saved to '{output_file}'")
print("🧹 Quitting browser...")
driver.quit()


Starting multi-region scraping...

🌎 Navigating to region: https://www.google.com/maps/search/%22Sportsman's+Warehouse%22+Alaska/@61.6390309,-147.7070618,6z?authuser=0&hl=en&entry=ttu&g_ep=EgoyMDI1MDYwMy4wIKXMDSoASAFQAw%3D%3D

🌎 Navigating to region: https://www.google.com/maps/search/%22Sportsman's+Warehouse%22+Alaska/@59.1998449,-135.2815249,7z?authuser=0&hl=en&entry=ttu&g_ep=EgoyMDI1MDYwNC4wIKXMDSoASAFQAw%3D%3D

🌎 Navigating to region: https://www.google.com/maps/search/%22Sportsman's+Warehouse%22/@48.1065043,-122.9546184,10z?authuser=0&hl=en&entry=ttu&g_ep=EgoyMDI1MDYwMy4wIKXMDSoASAFQAw%3D%3D

🌎 Navigating to region: https://www.google.com/maps/search/%22Sportsman's+Warehouse%22+VA/@38.209683,-80.9469872,8z?authuser=0&hl=en&entry=ttu&g_ep=EgoyMDI1MDYwMy4wIKXMDSoASAFQAw%3D%3D


SyntaxError: 'continue' not properly in loop (<ipython-input-7-3490244468>, line 60)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


The following code *SHOULD BE*** the updated code to convert the HTML to CSV - it inclydes the code to make the location column= the city of the store the review came from.....